In [0]:
import json, pprint, requests

# Helper to pretty print json
def pprint_j(i):
    print(json.dumps(i, indent=4, sort_keys=True))

class DatabricksRestClient:
    """A class to define wrappers for the REST API"""

    def __init__(self, token="ABCDEFG1234", url="https://myenv.cloud.databricks.com"):
        self.__token = {'Authorization': 'Bearer {0}'.format(token)}
        self.__url = url
        
    def get(self, endpoint, json_params=None, version='2.0', print_json=False):
        if version:
            ver = version
        full_endpoint = self.__url + '/api/{0}'.format(ver) + endpoint
        print(full_endpoint)
        if json_params:
            raw_results = requests.get(full_endpoint, headers=self.__token, params=json_params)
            results = raw_results.json()
        else:
            raw_results = requests.get(full_endpoint, headers=self.__token)
            results = raw_results.json()
        if print_json:
            print(json.dumps(results, indent=4, sort_keys=True))
        results['http_status_code'] = raw_results.status_code
        return results
    
    def post(self, endpoint, json_params = {}, printJson = False):
        if json_params:
            raw_results = requests.post(self.__url + '/api/2.0' + endpoint, headers=self.__token, json=json_params)
            results = raw_results.json()
        else: 
            print("Must have a payload in json_args param.")
            return {}
        if printJson:
            print(json.dumps(results, indent=4, sort_keys=True))
        # if results are empty, let's return the return status
        if results:
            results['http_status_code'] = raw_results.status_code
            return results
        else:
            return {'http_status_code': raw_results.status_code}
        
    def patch(self, endpoint, json_params = {}, printJson = False):
        if json_params:
            raw_results = requests.patch(self.__url + '/api/2.0' + endpoint, headers=self.__token, json=json_params)
            results = raw_results.json()
        else: 
            print("Must have a payload in json_args param.")
            return {}
        if printJson:
            print(json.dumps(results, indent=4, sort_keys=True))
        # if results are empty, let's return the return status
        if results:
            results['http_status_code'] = raw_results.status_code
            return results
        else:
            return {'http_status_code': raw_results.status_code}

    def get_users_groups(self):
        users_root = self.get('/workspace/list', {'path': '/Users'}).get('objects', [])
        get_users_home = list(map(lambda x: x.get('path').split('/')[2], users_root))
        user_group_map = {}
        all_users = self.get('/preview/scim/v2/Users').get('Resources', [])
        for x in all_users:
            u_name = x.get('userName')
            if u_name in get_users_home:
                user_group_map[u_name] = x.get('groups')
        return user_group_map
    
    def get_group_perms(self, group_list):
        group_read_perms = list(map( lambda x: {'group_name': x.get('display'), 'permission_level': 'CAN_READ'}, group_list))
        perms_arg = { "access_control_list": group_read_perms }
        return perms_arg
                
    def set_users_home_dirs_read_only_by_group(self):
        users_root = self.get('/workspace/list', {'path': '/Users'}).get('objects', [])
        user_group_dict = self.get_users_groups()
        for x in users_root:
            user_home = x.get('path')
            u_name = user_home.split('/')[2]
            u_groups = user_group_dict.get(u_name, None)
            if u_groups:
                perms_arg = self.get_group_perms(u_groups)
                dir_id = x.get('object_id')
                api = f'/permissions/directories/{dir_id}'
                print(f'Updating {u_name} to be read by their own groups')
                response = self.patch(api, perms_arg)

In [0]:
url = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().getOrElse(None) 
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().getOrElse(None)

client = DatabricksRestClient(token, url)

In [0]:
client.set_users_home_dirs_read_only_by_group()